In [1]:
import numpy as np
from keras.models import load_model
from sklearn.model_selection import KFold
from time import time
from timeit import default_timer as timer
import pickle


Using TensorFlow backend.


In [2]:
dataset = np.load('all_dataset_window19Middle.npy') # load

In [3]:
windowSize = 19
predictionIndex = 9
classSize = 8  # 2 or 3 
numberOfFeatures = 50 #44 

amino_acid_residues = 21
num_classes = 8

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv1D, AveragePooling1D, MaxPooling1D, TimeDistributed, LeakyReLU, BatchNormalization, Flatten
from keras import optimizers, callbacks
from keras import regularizers
from keras.utils import plot_model
import os

LR = 0.0005
drop_out = 0.3
batch_dim = 64
nn_epochs = 10
w_reg = regularizers.l2(0.0001)
number_filters = 16


def get_model():
   

    loss = 'categorical_crossentropy'


    m = Sequential()

#first convolutional neural netwok
    m.add(Conv1D( 16 , 3,  strides=1, padding='same', activation='relu', use_bias=True, input_shape=(windowSize, 21), kernel_regularizer=w_reg))
    m.add(BatchNormalization())

    m.add(Conv1D( 16, 3,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())
    m.add(Conv1D( 16, 5,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())
    m.add(Conv1D( 16, 7,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())


    m.add(Conv1D( 16, 3,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())
    m.add(Conv1D( 16, 5,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())
    m.add(Conv1D( 16, 7,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())

    m.add(Conv1D( 16, 3,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())
    m.add(Conv1D( 16, 5,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())
    m.add(Conv1D( 16, 7,  strides=1, padding='same', activation='relu', use_bias=True, kernel_regularizer=w_reg))
    m.add(BatchNormalization())


    m.add(Flatten())

    #4 dense layer
    m.add(Dense(200, activation='relu', use_bias=True,  kernel_regularizer=w_reg))

    #5 softmax output layer
    m.add(Dense(num_classes, activation = 'softmax'))

    opt = optimizers.Adam(lr=LR)
    m.compile(optimizer=opt, loss=loss,metrics=['accuracy', 'mae'])

    print("\nHyper Parameters\n")
    print("Learning Rate: " + str(LR))
    print("Drop out: " + str(drop_out))

    print("Batch dim: " + str(batch_dim))
    print("Number of epochs: " + str(nn_epochs))
    print("Regularizers: " + str(w_reg.l2))
    print("\nLoss: " + loss + "\n")
    m.summary()

    #os.environ["PATH"] += os.pathsep + 'C:/Users/Ieremie/Anaconda3/pkgs/graphviz-2.38-hfd603c8_2/Library/bin'
    #plot_model(m)
    
    return m
    

In [5]:
from sklearn.utils import shuffle
dataset = shuffle(dataset, random_state=111)
print(dataset.shape)
print(dataset[0,:2,:])

(1133502, 19, 50)
[[1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.99043465
  0.02297737 0.2158528  0.77729988 0.02412702 0.19309869 0.08706578
  0.01964677 0.22270013 0.0511737  0.04148712 0.12346705 0.03768789
  0.38936076 0.86875552 0.82200629 0.1665886  0.04565117 0.00364771
  0.5        0.11204704]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         1.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.         0.         0.33403307
  0.94479948 0.00151625 0.00482072 0.55477923 0.00196558 0.00910306
  0.90114391 0.00436397 0.98507971 0.97916371 0.00477298 0.00290041
 

In [ ]:
kf = KFold(n_splits=10, random_state=10, shuffle=True)
for train, test in kf.split(dataset):
    print(train,test)

In [6]:
l = 0
r = 113350
n = 1133502

fold = 0
valid_indexes = [()]


for i in range (10):
  
    fold += 1
    print(fold)
    
    m = get_model()
    start_time = timer()
    history = m.fit(np.concatenate((dataset[0:l], dataset[r:n]), axis=0)[:,:,0:21], np.concatenate((dataset[0:l], dataset[r:n]), axis=0)[:, 0, 21 : (21+classSize)],
                    epochs=nn_epochs, validation_data=(dataset[l:r][:,:,0:21], dataset[l:r][:, 0, 21 : (21+classSize)]) ,shuffle=True,batch_size=batch_dim)
    end_time = timer()
    print("\n\nTime elapsed: " + "{0:.2f}".format((end_time - start_time)) + " s")
    
    scores = m.evaluate(dataset[l:r][:,:,0:21], dataset[l:r][:, 0, 21 : (21+classSize)])
    print("Loss: " + str(scores[0]) + ", Accuracy: " + str(scores[1]) + ", MAE: " + str(scores[2]) + ' fold ' + str(fold))
    
    
    m.save('model_kfold' + str(fold) + '.h5')  # creates a HDF5 file 
    with open('model_scores_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(scores, file_pi)
    with open('model_history_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
    
    valid_indexes.append((l, r))
    r += 113350
    l += 113350

with open('model_kfold_indexes', 'wb') as file_pi:
    pickle.dump(valid_indexes, file_pi)   

1

Hyper Parameters

Learning Rate: 0.0005
Drop out: 0.3
Batch dim: 64
Number of epochs: 10
Regularizers: 1e-04

Loss: categorical_crossentropy

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 19, 16)            1024      
_________________________________________________________________
batch_normalization_1 (Batch (None, 19, 16)            64        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 19, 16)            784       
_________________________________________________________________
batch_normalization_2 (Batch (None, 19, 16)            64        
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 19, 16)            1296      
_________________________________________________________________
batch_normalization_3 (Batch (None, 19, 1

KeyboardInterrupt: 

In [ ]:
kf = KFold(n_splits=10, random_state=10, shuffle=True)
fold = 0
valid_indexes = [([],[])]
for train, test in kf.split(dataset):

    fold += 1
    print(fold)
    
    m = get_model()
    start_time = timer()
    history = m.fit(dataset[train][:,:,0:21], dataset[train][:,:,21 : (21+classSize)], epochs=nn_epochs,
                    batch_size=batch_dim, validation_data=(dataset[test][:,:,0:21], dataset[test][:,:,21 : (21+classSize)]),shuffle=True)
    end_time = timer()
    print("\n\nTime elapsed: " + "{0:.2f}".format((end_time - start_time)) + " s")

    
    scores = m.evaluate(dataset[test][:,:,0:21], dataset[test][:,:,21 : (21+classSize)])
    print("Loss: " + str(scores[0]) + ", Accuracy: " + str(scores[1]) + ", MAE: " + str(scores[2]) + ' fold ' + str(fold))
    
    
    m.save('model_kfold' + str(fold) + '.h5')  # creates a HDF5 file 
    with open('model_scores_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(scores, file_pi)
    with open('model_history_kfold' + str(fold), 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
    
    valid_indexes.append((train, test))

with open('model_kfold_indexes', 'wb') as file_pi:
    pickle.dump(valid_indexes, file_pi)    
    